# Importing Packages

In [1]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True



%matplotlib qt



[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data

In [34]:

data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)



galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 



galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

mangaid=galaxies.loc[:,'mangaid']

plateifu=galaxies.loc[:,'plateifu']

if plateifu.all()==np.unique(plateifu).all() and mangaid.all()==np.unique(mangaid.all()):
    print('Is Unique List of Galaxies')
    
else:
    print('Try Again')


z=galaxies.loc[:,'z']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)

bars=np.linspace(np.min(log_mass),11.25,num=10)
plt.hist(log_mass)
plt.vlines(bars,ymin=0,ymax=800)


mass_cat = np.zeros_like(log_mass)
mass_cat[(log_mass> bars[0]) & (log_mass<bars[1])]=int(0)
mass_cat[(log_mass> bars[1]) & (log_mass<bars[2])]=int(1)
mass_cat[(log_mass> bars[2]) & (log_mass<bars[3])]=int(2)
mass_cat[(log_mass> bars[3]) & (log_mass<bars[4])]=int(3)
mass_cat[(log_mass> bars[4]) & (log_mass<bars[5])]=int(4)
mass_cat[(log_mass> bars[5]) & (log_mass<bars[6])]=int(5)
mass_cat[(log_mass> bars[6]) & (log_mass<bars[7])]=int(6)
mass_cat[(log_mass> bars[7]) & (log_mass<bars[8])]=int(7)
mass_cat[(log_mass> bars[8]) & (log_mass<bars[9])]=int(8)
mass_cat[log_mass> bars[9]] = int(9)


print(np.mean(mass_cat))
all_galaxies=np.arange(0,len(log_mass))
class1=np.where((log_mass> bars[0]) & (log_mass<bars[1]))
class10=np.where(log_mass> bars[9])

Is Unique List of Galaxies
4.226525494566732


# Importing and Pooling Grad-CAM Maps

In [39]:
all_maps=[]
for j in range (len(all_galaxies)):
    i=all_galaxies[j]
    all_maps.append(np.loadtxt('/home/juanp/Documents/Grad-CAM Results/Class 10 Focus/Conv3/MapMap '+str(i)))


In [40]:
map_avg=sum(all_maps)/len(all_maps)

plt.imshow(map_avg)
plt.colorbar()
